In [2]:
!pip install -q transformers torch datasets pandas scikit-learn accelerate bitsandbytes peft fuzzywuzzy python-levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
import re
import logging
from fuzzywuzzy import fuzz
import warnings
warnings.filterwarnings('ignore')

In [6]:
train_df = pd.read_csv('data/train.csv')
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text.strip())
    text = re.sub(r'[^\w\s.,!?—–-]', ' ', text)
    if len(text) < 10:
        return ""
    return text

train_df['text_clean'] = train_df['text'].apply(preprocess_text)
train_df = train_df[train_df['text_clean'] != ''].reset_index(drop=True)
print(f"Размер после предобработки: {len(train_df)}")
categories = ['бытовая техника', 'обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей', 'украшения и аксессуары', 'электроника', 'нет товара']

Размер после предобработки: 1803


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

# Модель 1: mGPT для первичной разметки
model_name1 = "ai-forever/mGPT"
tokenizer1 = AutoTokenizer.from_pretrained(model_name1)
tokenizer1.pad_token = tokenizer1.eos_token
base_model1 = AutoModelForCausalLM.from_pretrained(model_name1, quantization_config=quant_config, device_map="auto", trust_remote_code=False, torch_dtype=torch.float16)

# Модель 2: RuGPT3Large для перепроверки
model_name2 = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
tokenizer2.pad_token = tokenizer2.eos_token
base_model2 = AutoModelForCausalLM.from_pretrained(model_name2, torch_dtype=torch.float16).to(device)
print(f"Модели загружены на {device}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Модели загружены на cuda


In [10]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Словарь ключевых слов для страховки
keyword_map = {
    "одежда": ["футболка", "пуховик", "блузка", "юбка", "штаны", "джинсы", "свитер", "куртка", "платье", "кофта", "шорты", "рубашка", "комбинезон", "халат", "кардиган", "майка", "туника", "водолазка", "жилет", "пижама", "толстовка", "пальто", "брюки", "топ", "сарафан", "ветровка", "боди", "джемпер", "свитшот", "комбез", "трусы", "колготки"],
    "обувь": ["кроссовки", "туфли", "ботинки", "сапоги", "тапки", "сандалии", "кеды", "мокасины", "балетки", "босоножки", "сланцы", "кроссы"],
    "бытовая техника": ["холодильник", "микроволновка", "пылесос", "стиральная машина", "чайник", "утюг", "блендер", "фен", "плита", "духовка", "тостер"],
    "посуда": ["тарелки", "кружка", "чашка", "ложка", "вилка", "нож", "кастрюля", "сковорода", "бокал", "чайник керамический", "стакан", "миска", "салатник"],
    "текстиль": ["полотенце", "шторы", "постельное", "одеяло", "подушка", "скатерть", "простыня", "наволочка", "покрывало", "ковёр", "занавеска"],
    "товары для детей": ["игрушка", "погремушка", "пеленка", "коляска", "кроватка", "памперс", "подгузник", "автокресло"],
    "украшения и аксессуары": ["серьги", "браслет", "кольцо", "цепочка", "кулон", "часы", "ремень", "шарф", "сумка", "кошелёк", "очки", "шапка", "носочки", "перчатки"],
    "электроника": ["телефон", "наушники", "планшет", "зарядка", "чехол", "кабель", "ноутбук", "клавиатура", "смартфон", "гарнитура", "колонка", "монитор", "мышка", "аккумулятор"],
    "нет товара": ["доставка", "спор", "возврат", "не пришёл", "не дошло", "деньги", "задержка", "отслеживание", "курьер", "упаковка", "заказ", "продавец"]
}

def keyword_check(text):
    text_lower = text.lower()
    for category, keywords in keyword_map.items():
        if any(keyword in text_lower for keyword in keywords):
            logger.info(f"Ключевое слово найдено: {category} для '{text[:50]}...'")
            return category
    return None

# Промпт для первичной модели (CoT + больше примеров для редких классов)
few_shot_primary = """Ты классификатор отзывов по категориям товаров. Выбери одну категорию из: {categories}.
Шаги:
1. Ищи прямые упоминания товара (футболка → одежда, кроссовки → обувь).
2. Если нет прямых, анализируй косвенные признаки (ткань/швы → одежда, нагревается → посуда/бытовая техника, доставка → нет товара).
3. Выведи только категорию.

Примеры:
1. Отзыв: "Футболка велика, ткань синтетика." Категория: одежда
2. Отзыв: "Кроссовки жмут, но удобные." Категория: обувь
3. Отзыв: "Холодильник шумит, но охлаждает." Категория: бытовая техника
4. Отзыв: "Тарелка разбилась при доставке." Категория: посуда
5. Отзыв: "Полотенце линяет, но мягкое." Категория: текстиль
6. Отзыв: "Игрушка сломалась быстро." Категория: товары для детей
7. Отзыв: "Серьги потемнели за неделю." Категория: украшения и аксессуары
8. Отзыв: "Телефон быстро садится." Категория: электроника
9. Отзыв: "Заказ не пришёл, спор не помог." Категория: нет товара
10. Отзыв: "Ткань тонкая, швы кривые." Категория: одежда (косвенно про одежду)
11. Отзыв: "Нагревается быстро, ручка хлипкая." Категория: посуда (косвенно про посуду)
12. Отзыв: "Доставка три месяца, трек не отслеживался." Категория: нет товара
13. Отзыв: "Тяжёлый, но мощный." Категория: бытовая техника (косвенно)
14. Отзыв: "Звук плохой, быстро сломались." Категория: электроника (косвенно)
15. Отзыв: "Шапка холодная, но стильная." Категория: украшения и аксессуары

Отзыв: "{text}"
Категория:"""

def classify_primary(text, model, tokenizer, categories):
    # Сначала проверяем ключевые слова
    keyword_result = keyword_check(text)
    if keyword_result:
        return keyword_result

    # Если ключевых слов нет, используем модель
    prompt = few_shot_primary.format(categories=', '.join(categories), text=text)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10, temperature=0.05, do_sample=True)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Категория:")[-1].strip().lower()
    best_match = max(categories, key=lambda cat: fuzz.ratio(pred, cat.lower()))
    logger.info(f"Модель (первичная): {best_match} для '{text[:50]}...'")
    return best_match

# Промпт для перепроверки
few_shot_secondary = """Ты проверяешь разметку отзывов по категориям: {categories}. Шаги:
1. Прочитай отзыв и первичный label.
2. Если первичный неверен (например, "одежда" для доставки или "нет товара" для явного товара), исправь.
3. Учитывай косвенные признаки (ткань → одежда, нагревается → посуда, доставка → нет товара).
4. Выведи только правильную категорию.

Примеры:
1. Первичная: нет товара. Отзыв: "Футболка велика." Правильная: одежда
2. Первичная: одежда. Отзыв: "Заказ не пришёл." Правильная: нет товара
3. Первичная: одежда. Отзыв: "Платье тонкое." Правильная: одежда
4. Первичная: нет товара. Отзыв: "Ткань липнет, швы кривые." Правильная: одежда
5. Первичная: бытовая техника. Отзыв: "Чашка нагревается." Правильная: посуда
6. Первичная: одежда. Отзыв: "Кроссовки износились." Правильная: обувь
7. Первичная: нет товара. Отзыв: "Телефон зависает." Правильная: электроника
8. Первичная: нет товара. Отзыв: "Доставка три месяца." Правильная: нет товара

Отзыв: "{text}"
Первичная категория: "{primary}"
Правильная категория:"""

def check_secondary(text, primary, model, tokenizer, categories):
    # Проверка ключевых слов
    keyword_result = keyword_check(text)
    if keyword_result and keyword_result != primary and keyword_result != "нет товара":
        logger.info(f"Ключевое слово перебивает: {keyword_result} вместо {primary} для '{text[:50]}...'")
        return keyword_result

    # Если ключевых слов нет или они для "нет товара", проверяем моделью
    prompt = few_shot_secondary.format(categories=', '.join(categories), text=text, primary=primary)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10, temperature=0.05, do_sample=True)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Правильная категория:")[-1].strip().lower()
    best_match = max(categories, key=lambda cat: fuzz.ratio(pred, cat.lower()))

    # Редактирование, если расхождение >40%
    if fuzz.ratio(pred, primary.lower()) < 60:
        logger.info(f"Вторичная модель изменила: {best_match} вместо {primary} для '{text[:50]}...'")
        return best_match
    logger.info(f"Вторичная модель сохранила: {primary} для '{text[:50]}...'")
    return primary

In [11]:
batch_size = 20
train_df['label_primary'] = ''
train_df['label_final'] = ''

start_time = time.time()
for i in range(0, len(train_df), batch_size):
    batch_df = train_df.iloc[i:i+batch_size]

    # Первичная разметка
    for idx in batch_df.index:
        text = train_df.at[idx, 'text_clean']
        primary = classify_primary(text, base_model1, tokenizer1, categories)
        train_df.at[idx, 'label_primary'] = primary

    # Перепроверка
    for idx in batch_df.index:
        text = train_df.at[idx, 'text_clean']
        primary = train_df.at[idx, 'label_primary']
        final = check_secondary(text, primary, base_model2, tokenizer2, categories)
        train_df.at[idx, 'label_final'] = final

    elapsed_batch = time.time() - start_time
    print(f"Батч {i//batch_size + 1}: {len(batch_df)} примеров, ср. время {elapsed_batch / len(batch_df):.2f}с/пример")
    torch.cuda.empty_cache()

# Сохранение
train_df[['text_clean', 'label_final']].to_csv('train_labeled.csv', index=False)
print(f"Разметка завершена. Общее время: {time.time() - start_time:.2f}с")
print("Распределение первичных labels:")
print(train_df['label_primary'].value_counts())
print("\nРаспределение финальных labels:")
print(train_df['label_final'].value_counts())

# Проверка первых 5 строк
print("\nПервые 5 строк:")
print(train_df[['text_clean', 'label_primary', 'label_final']].head(5))

Батч 1: 20 примеров, ср. время 0.34с/пример
Батч 2: 20 примеров, ср. время 0.71с/пример
Батч 3: 20 примеров, ср. время 1.16с/пример
Батч 4: 20 примеров, ср. время 1.63с/пример
Батч 5: 20 примеров, ср. время 1.93с/пример
Батч 6: 20 примеров, ср. время 2.25с/пример
Батч 7: 20 примеров, ср. время 2.62с/пример
Батч 8: 20 примеров, ср. время 2.98с/пример
Батч 9: 20 примеров, ср. время 3.30с/пример
Батч 10: 20 примеров, ср. время 3.67с/пример
Батч 11: 20 примеров, ср. время 4.05с/пример
Батч 12: 20 примеров, ср. время 4.35с/пример
Батч 13: 20 примеров, ср. время 4.77с/пример
Батч 14: 20 примеров, ср. время 5.13с/пример
Батч 15: 20 примеров, ср. время 5.48с/пример
Батч 16: 20 примеров, ср. время 5.88с/пример
Батч 17: 20 примеров, ср. время 6.25с/пример
Батч 18: 20 примеров, ср. время 6.74с/пример
Батч 19: 20 примеров, ср. время 7.10с/пример
Батч 20: 20 примеров, ср. время 7.49с/пример
Батч 21: 20 примеров, ср. время 7.84с/пример
Батч 22: 20 примеров, ср. время 8.18с/пример
Батч 23: 20 примеро

In [12]:
label_counts = train_df['label_final'].value_counts()
rare_classes = [cat for cat in categories if label_counts.get(cat, 0) < 100]  # <5.5%

if rare_classes:
    print(f"Редкие классы: {rare_classes}. Генерируем 100 синтетических на класс.")
    augmented = []
    for cat in rare_classes:
        for _ in range(100):
            prompt = f"Генерируй короткий отзыв на русском о товаре из категории '{cat}'. Пример: 'Футболка мягкая, но велика.'"
            inputs = tokenizer2(prompt, return_tensors="pt", truncation=True, max_length=256).to(device)
            with torch.no_grad():
                outputs = base_model2.generate(**inputs, max_new_tokens=50, temperature=0.7, do_sample=True)
            synth_text = tokenizer2.decode(outputs[0], skip_special_tokens=True).split("Пример:")[-1].strip()
            augmented.append({'text_clean': synth_text, 'label_final': cat})

    aug_df = pd.DataFrame(augmented)
    train_df = pd.concat([train_df, aug_df], ignore_index=True)
    train_df.to_csv('train_labeled_augmented.csv', index=False)
    print("Аугментированный датасет сохранён как 'train_labeled_augmented.csv'")
    print("Новое распределение:")
    print(train_df['label_final'].value_counts())

Редкие классы: ['бытовая техника', 'обувь', 'посуда', 'текстиль', 'электроника']. Генерируем 100 синтетических на класс.
Аугментированный датасет сохранён как 'train_labeled_augmented.csv'
Новое распределение:
label_final
одежда                    1232
нет товара                 196
электроника                175
бытовая техника            141
украшения и аксессуары     127
товары для детей           121
обувь                      108
текстиль                   102
посуда                     101
Name: count, dtype: int64
